In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
matches = pd.read_csv("results.csv")
rankings = pd.read_csv("fifa_ranking.csv")
world_cup_matches = pd.read_csv("World Cup 2018 Dataset.csv")
players = pd.read_csv("FullData.csv")
all_time_stats = pd.read_csv("all_time_fifa_statistics.csv")

In [4]:
rankings = rankings.loc[:,['rank', 
                           'country_full', 
                           'country_abrv', 
                           'cur_year_avg_weighted', 
                           'rank_date', 
                           'two_year_ago_weighted', 
                           'three_year_ago_weighted']]
rankings = rankings.replace({"IR Iran": "Iran"})
rankings['weighted_points'] =  rankings['cur_year_avg_weighted'] + rankings['two_year_ago_weighted'] + rankings['three_year_ago_weighted']
rankings["rank_date"] = pd.to_datetime(rankings["rank_date"])

In [5]:
matches = matches.replace({"Germany DR": "Germany", "China": "China PR"})
matches["date"] = pd.to_datetime(matches["date"])

In [6]:
world_cup_matches = world_cup_matches.loc[:, ['Team', 
                                              'Group', 
                                              'First match \nagainst', 
                                              'Second match\n against', 
                                              'Third match\n against']]
world_cup_matches = world_cup_matches.dropna(how='all')
world_cup_matches = world_cup_matches.replace({"IRAN": "Iran", 
                               "Costarica": "Costa Rica", 
                               "Porugal": "Portugal", 
                               "Columbia": "Colombia", 
                               "Korea" : "Korea Republic"})
world_cup_matches = world_cup_matches.set_index('Team')

In [7]:
players = players.loc[:, ["Nationality",
                            "Rating",
                            "Age",
                            "Weak_foot",
                            "Skill_Moves",
                            "Ball_Control",
                            "Dribbling",
                            "Marking",
                            "Sliding_Tackle",
                            "Standing_Tackle",
                            "Aggression",
                            "Reactions",
                            "Attacking_Position",
                            "Interceptions",
                            "Vision",
                            "Composure",
                            "Crossing",
                             "Short_Pass",
                             "Long_Pass",
                             "Acceleration",
                             "Speed",
                             "Stamina",
                             "Strength",
                             "Balance",
                             "Agility",
                             "Jumping",
                             "Heading",
                             "Shot_Power",
                             "Finishing",
                             "Long_Shots",
                             "Curve",
                             "Freekick_Accuracy",
                             "Penalties",
                             "Volleys"]]
players = players.dropna(how="all")
grouped = players.groupby(["Nationality"], as_index = False)
players = grouped.aggregate(np.mean)


In [8]:
matches.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


In [9]:
rankings.head()

,rank,country_full,country_abrv,cur_year_avg_weighted,rank_date,two_year_ago_weighted,three_year_ago_weighted,weighted_points
0,1,Germany,GER,0.0,1993-08-08,0.0,0.0,0.0
1,2,Italy,ITA,0.0,1993-08-08,0.0,0.0,0.0
2,3,Switzerland,SUI,0.0,1993-08-08,0.0,0.0,0.0
3,4,Sweden,SWE,0.0,1993-08-08,0.0,0.0,0.0
4,5,Argentina,ARG,0.0,1993-08-08,0.0,0.0,0.0


In [10]:
# I want to have the ranks for every day 
rankings = rankings.set_index(['rank_date'])\
            .groupby(['country_full'], group_keys=False)\
            .resample('D').first()\
            .fillna(method='ffill')\
            .reset_index()

# join the ranks
matches = matches.merge(rankings, 
                        left_on=['date', 'home_team'], 
                        right_on=['rank_date', 'country_full'])
matches = matches.merge(rankings, 
                        left_on=['date', 'away_team'], 
                        right_on=['rank_date', 'country_full'], 
                        suffixes=('_home', '_away'))

In [11]:
matches = matches.merge(players,
                       left_on =["home_team"],
                       right_on = ["Nationality"])

matches = matches.merge(players,
                        left_on = ['away_team'],
                        right_on = ["Nationality"],
                        suffixes = ('_home', "_away"))

In [12]:
matches = matches.merge(all_time_stats,
                       left_on = ["home_team"],
                       right_on = ["Country"])

matches = matches.merge(all_time_stats,
                       left_on = ["away_team"],
                        right_on = ["Country"],
                       suffixes = ("_home", "_away"))

In [13]:
matches['rank_difference'] = matches['rank_home'] - matches['rank_away']
matches['average_rank'] = (matches['rank_home'] + matches['rank_away'])/2
matches['score_difference'] = matches['home_score'] - matches['away_score']
matches["point_difference"] = matches['weighted_points_home'] - matches['weighted_points_away']
matches["rating_difference"] = matches["Rating_home"] - matches["Rating_away"]
matches["Age_difference"] = matches["Age_home"] - matches["Age_away"]
matches["Weak_foot_difference"] = matches["Weak_foot_home"] - matches["Weak_foot_away"]
matches["Skill_Moves_difference"] = matches["Skill_Moves_home"] - matches["Skill_Moves_away"]
matches["Ball_Control_difference"] = matches["Ball_Control_home"] - matches["Ball_Control_away"]
matches["Dribbling_difference"] = matches["Dribbling_home"] - matches["Dribbling_away"]
matches["Marking_difference"] = matches["Marking_home"] - matches["Marking_away"]
matches["Sliding_Tackle_difference"] = matches["Sliding_Tackle_home"] - matches["Sliding_Tackle_away"]
matches["Standing_Tackle_difference"] = matches["Standing_Tackle_home"] - matches["Standing_Tackle_away"]
matches["Aggression_difference"] = matches["Aggression_home"] - matches["Aggression_away"]
matches["Reactions_difference"] = matches["Reactions_home"] - matches["Reactions_away"]
matches["Attacking_Position_difference"] = matches["Attacking_Position_home"] - matches["Attacking_Position_away"]
matches["Interceptions_difference"] = matches["Interceptions_home"] - matches["Interceptions_away"]
matches["Vision_difference"] = matches["Vision_home"] - matches["Vision_away"]
matches["Composure_difference"] = matches["Composure_home"] - matches["Composure_away"]
matches["Crossing_difference"] = matches["Crossing_home"] - matches["Crossing_away"]
matches["Short_Pass_difference"] = matches["Short_Pass_home"] - matches["Short_Pass_away"]
matches["Long_Pass_difference"] = matches["Long_Pass_home"] - matches["Long_Pass_away"]
matches["Stamina_difference"] = matches["Stamina_home"] - matches["Stamina_away"]
matches["Penalties_difference"] = matches["Penalties_home"] - matches["Penalties_away"]
matches["Acceleration_difference"] = matches["Acceleration_home"] - matches["Acceleration_away"]
matches["Speed_difference"] = matches["Speed_home"] - matches["Speed_away"]
matches["Strength_difference"] = matches["Strength_home"] - matches["Strength_away"]
matches["Balance_difference"] = matches["Balance_home"] - matches["Balance_away"]
matches["Agility_difference"] = matches["Agility_home"] - matches["Agility_away"]
matches["Jumping_difference"] = matches["Jumping_home"] - matches["Jumping_away"]
matches["Heading_difference"] = matches["Heading_home"] - matches["Heading_away"]
matches["Shot_Power_difference"] = matches["Shot_Power_home"] - matches["Shot_Power_away"]
matches["Finishing_difference"] = matches["Finishing_home"] - matches["Finishing_away"]
matches["Long_Shots_difference"] = matches["Long_Shots_home"] - matches["Long_Shots_away"]
matches["Curve_difference"] = matches["Curve_home"] - matches["Curve_away"]
matches["Freekick_Accuracy_difference"] = matches["Freekick_Accuracy_home"] - matches["Freekick_Accuracy_away"]
matches["Volleys_difference"] = matches["Volleys_home"] - matches["Volleys_away"]
matches["Part's_difference"] = matches["Part's_home"] - matches["Part's_away"]
matches["Played_difference"] = matches["Played_home"] - matches["Played_away"]
matches["Won_difference"] = matches["Won_home"] - matches["Won_away"]
matches["Drawn_difference"] = matches["Drawn_home"] - matches["Drawn_away"]
matches["Lost_difference"] = matches["Lost_home"] - matches["Lost_away"]
matches["Goal_Difference_difference"] = matches["Goal Difference_home"] - matches["Goal Difference_away"]
matches["Points_difference"] = matches["Points_home"] - matches["Points_away"]
matches["Average_points_difference"] = matches["Average_points_home"] - matches["Average_points_away"]
matches['is_won'] = matches['score_difference'] > 0 # take draw as lost
matches['is_stake'] = matches['tournament'] != 'Friendly'


In [14]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score

X = matches.loc[:,['average_rank', 
                    'rank_difference', 
                    "point_difference", 
                    'is_stake', 
                    "rating_difference", 
                     "Age_difference",
                    "Weak_foot_difference",
                     "Skill_Moves_difference",
                    "Ball_Control_difference",
                     "Dribbling_difference",
                     "Marking_difference",
                     "Sliding_Tackle_difference",
                     "Standing_Tackle_difference",
                     "Aggression_difference",
                     "Reactions_difference",
                     "Interceptions_difference",
                     "Vision_difference",
                   "Crossing_difference",
                     "Short_Pass_difference",
                     "Long_Pass_difference",
                    "Stamina_difference",
                     "Penalties_difference",
                     "Acceleration_difference",                   
                     "Speed_difference",
                    "Strength_difference",
                    "Balance_difference",
                     "Agility_difference",
                     "Jumping_difference",
                    "Heading_difference",
                     "Shot_Power_difference",
                    "Finishing_difference",
                   "Long_Shots_difference",
                     "Curve_difference",
                    "Freekick_Accuracy_difference",
                     "Volleys_difference",
                     "Won_difference",
                     "Drawn_difference",
                     "Lost_difference",
                     "Average_points_difference",
                  ]]
y = matches['is_won']

In [15]:
y = pd.get_dummies(y, drop_first = True)
y.head()

,True
0,1
1,1
2,1
3,0
4,0


In [16]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6057 entries, 0 to 6056
Data columns (total 39 columns):
average_rank                    6057 non-null float64
rank_difference                 6057 non-null float64
point_difference                6057 non-null float64
is_stake                        6057 non-null bool
rating_difference               6057 non-null float64
Age_difference                  6057 non-null float64
Weak_foot_difference            6057 non-null float64
Skill_Moves_difference          6057 non-null float64
Ball_Control_difference         6057 non-null float64
Dribbling_difference            6057 non-null float64
Marking_difference              6057 non-null float64
Sliding_Tackle_difference       6057 non-null float64
Standing_Tackle_difference      6057 non-null float64
Aggression_difference           6057 non-null float64
Reactions_difference            6057 non-null float64
Interceptions_difference        6057 non-null float64
Vision_difference               

In [17]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [18]:
# Feature Scaling
# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler()
# X_train = sc.fit_transform(X_train)
# X_test = sc.transform(X_test)

In [19]:
# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense

/Applications/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [20]:
# Initialising the ANN
classifier_ann = Sequential()

# Adding the input layer and the first hidden layer
classifier_ann.add(Dense(activation="relu", input_dim=39, units=5, kernel_initializer="uniform"))

# Adding the second hidden layer
classifier_ann.add(Dense(units = 5, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifier_ann.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier_ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
classifier_ann.fit(X_train, y_train, batch_size = 10, epochs = 10)

Epoch 1/10
4845/4845 [==============================] - 1s 277us/step - loss: 0.6511 - acc: 0.6175
Epoch 2/10
4845/4845 [==============================] - 1s 252us/step - loss: 0.6329 - acc: 0.6411
Epoch 3/10
4845/4845 [==============================] - 1s 260us/step - loss: 0.6311 - acc: 0.6438
Epoch 4/10
4845/4845 [==============================] - 1s 249us/step - loss: 0.6305 - acc: 0.6446
Epoch 5/10
4845/4845 [==============================] - 1s 256us/step - loss: 0.6295 - acc: 0.6433
Epoch 6/10
4845/4845 [==============================] - 1s 218us/step - loss: 0.6292 - acc: 0.6460
Epoch 7/10
4845/4845 [==============================] - 1s 198us/step - loss: 0.6279 - acc: 0.6481
Epoch 8/10
4845/4845 [==============================] - 1s 214us/step - loss: 0.6274 - acc: 0.6483
Epoch 9/10
4845/4845 [==============================] - 1s 211us/step - loss: 0.6270 - acc: 0.6431
Epoch 10/10
4845/4845 [==============================] - 1s 208us/step - loss: 0.6265 - acc: 0.6456


In [21]:
#logreg = linear_model.LogisticRegression()
#logreg = logreg.fit(X_train, y_train)

In [22]:
#y_pred_logit = logreg.predict(X_test)
y_pred_ann = classifier_ann.predict(X_test)

In [23]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred_logit)

NameError: name 'y_pred_logit' is not defined

In [ ]:
cm

In [ ]:
y_pred_ann

In [ ]:
y_pred_ann = (y_pred_ann > 0.5)

In [ ]:
cm = confusion_matrix(y_test, y_pred_ann)
cm

In [24]:
fpr, tpr, _ = roc_curve(y_test, logreg.predict_proba(X_test)[:,1])
plt.figure(figsize=(15,5))
ax = plt.subplot(1,3,1)
ax.plot([0, 1], [0, 1], 'k--')
ax.plot(fpr, tpr)
ax.set_title('AUC score is {0:0.2}'.format(roc_auc_score(y_test, logreg.predict_proba(X_test)[:,1])))
ax.set_aspect(1)

ax = plt.subplot(1,3,2)
cm = confusion_matrix(y_test, logreg.predict(X_test))
ax.imshow(cm, cmap='Blues', clim = (0, cm.max())) 

ax.set_xlabel('Predicted label')
ax.set_title('Performance on the Test set')

ax = plt.subplot(1,3,3)
cm = confusion_matrix(y_train, logreg.predict(X_train))
ax.imshow(cm, cmap='Blues', clim = (0, cm.max())) 
ax.set_xlabel('Predicted label')
ax.set_title('Performance on the Training set')
pass

NameError: name 'logreg' is not defined

In [25]:
plt.show()

In [26]:
world_cup_rankings = rankings.loc[(rankings['rank_date'] == rankings['rank_date'].max()) & 
                                    rankings['country_full'].isin(world_cup_matches.index.unique())]


world_cup_rankings = world_cup_rankings.merge(players,
                                              left_on =["country_full"],
                                              right_on = ["Nationality"])


world_cup_rankings = world_cup_rankings.merge(all_time_stats,
                       left_on = ["country_full"],
                       right_on = ["Country"])


world_cup_rankings = world_cup_rankings.set_index(['country_full'])
world_cup_rankings

,rank_date,rank,country_abrv,cur_year_avg_weighted,two_year_ago_weighted,three_year_ago_weighted,weighted_points,Nationality,Rating,Age,...,Played,Won,Drawn,Lost,Goals For,Goals Against,Goal Difference,Points,Average_points,Best_finish
country_full,,,,,,,,,,,,,,,,,,,,,
Argentina,2018-06-07,5.0,ARG,404.07,248.99,183.59,836.65,Argentina,67.532361,26.384686,...,79,42,15,22,132,88,44,141,1.78,Winner (2 times)
Australia,2018-06-07,36.0,AUS,366.60,98.16,59.79,524.55,Australia,62.435897,24.504274,...,15,2,4,9,13,29,-16,10,0.66,Second round
Belgium,2018-06-07,3.0,BEL,629.98,158.94,186.58,975.50,Belgium,68.290566,24.362264,...,42,15,9,18,55,66,-11,54,1.29,Fourth place
Brazil,2018-06-07,2.0,BRA,558.95,168.06,162.38,889.39,Brazil,70.023887,27.786102,...,105,70,18,17,222,103,119,228,2.17,Winner (5 times)
Colombia,2018-06-07,16.0,COL,292.09,199.73,166.38,658.20,Colombia,65.305743,25.123311,...,19,7,2,10,27,29,-2,23,1.21,Quarter-finals
Costa Rica,2018-06-07,23.0,CRC,410.90,99.73,133.11,643.74,Costa Rica,69.100000,26.366667,...,16,5,4,7,17,24,-7,19,1.19,Quarter-finals
Croatia,2018-06-07,20.0,CRO,397.75,100.79,110.25,608.79,Croatia,68.431034,25.103448,...,18,9,2,7,26,17,9,29,1.61,Third place
Denmark,2018-06-07,12.0,DEN,723.72,90.07,80.21,894.00,Denmark,63.222222,24.780702,...,18,9,3,6,29,25,4,30,1.66,Quarter-finals
Egypt,2018-06-07,45.0,EGY,208.27,99.48,50.64,358.39,Egypt,68.766667,26.666667,...,6,0,2,4,4,10,-6,2,0.33,First round


In [27]:
X_test = pd.DataFrame(X_test)

In [28]:
from itertools import combinations

world_cup_matches['points'] = 0
world_cup_matches['total_prob'] = 0

for group in set(world_cup_matches['Group']):
    print('___Starting group {}:___'.format(group))
    for home, away in combinations(world_cup_matches.query('Group == "{}"'.format(group)).index, 2):
        print("{} vs. {}: ".format(home, away), end='')
        row = pd.DataFrame(np.array([[np.nan, 
                                      np.nan, 
                                      np.nan, 
                                      True, 
                                      np.nan,
                                      np.nan,
                                      np.nan,
                                      np.nan,
                                      np.nan,
                                      np.nan,
                                      np.nan,
                                      np.nan,
                                      np.nan,
                                      np.nan,
                                      np.nan,
                                      np.nan,
                                      np.nan,
                                      np.nan,
                                      np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan]]), columns= X_test.columns)
        
        
        home_rank = world_cup_rankings.loc[home, 'rank']
        home_points = world_cup_rankings.loc[home, 'weighted_points']
        home_rating = world_cup_rankings.loc[home,'Rating']
        home_age = world_cup_rankings.loc[home,'Age']
        home_weak_foot = world_cup_rankings.loc[home,'Weak_foot']
        home_Skill_Moves = world_cup_rankings.loc[home,'Skill_Moves']
        home_Ball_Control = world_cup_rankings.loc[home,'Ball_Control']
        home_Dribbling = world_cup_rankings.loc[home, 'Dribbling']
        home_Marking = world_cup_rankings.loc[home, 'Marking']
        home_Sliding_Tackle = world_cup_rankings.loc[home, 'Sliding_Tackle']
        home_Standing_Tackle = world_cup_rankings.loc[home, 'Standing_Tackle']
        home_Aggression = world_cup_rankings.loc[home, 'Aggression']
        home_Reactions = world_cup_rankings.loc[home, 'Reactions']
        home_Interceptions = world_cup_rankings.loc[home, 'Interceptions']
        home_Vision = world_cup_rankings.loc[home, "Vision"]
        home_Crossing = world_cup_rankings.loc[home, "Crossing"]
        home_Short_Pass = world_cup_rankings.loc[home, "Short_Pass"]
        home_Long_Pass = world_cup_rankings.loc[home, "Long_Pass"]                 
        home_stamina = world_cup_rankings.loc[home,'Stamina']
        home_Penalties = world_cup_rankings.loc[home,'Penalties']
        home_Acceleration = world_cup_rankings.loc[home,'Acceleration']        
        home_speed = world_cup_rankings.loc[home,'Speed']
        home_strength = world_cup_rankings.loc[home,'Strength']
        home_balance = world_cup_rankings.loc[home,'Balance']
        home_agility = world_cup_rankings.loc[home,'Agility']
        home_jumping = world_cup_rankings.loc[home,'Jumping']
        home_won = world_cup_rankings.loc[home,'Won']
        home_drawn = world_cup_rankings.loc[home,'Drawn']
        home_lost = world_cup_rankings.loc[home,'Lost']
        home_average_points = world_cup_rankings.loc[home,'Average_points']
        home_Heading = world_cup_rankings.loc[home,'Heading']
        home_Shot_Power = world_cup_rankings.loc[home,'Shot_Power']
        home_Finishing = world_cup_rankings.loc[home,'Finishing']
        home_Long_Shots = world_cup_rankings.loc[home,'Long_Shots']
        home_Curve = world_cup_rankings.loc[home,'Curve']
        home_Freekick_Accuracy = world_cup_rankings.loc[home,'Freekick_Accuracy']
        home_Volleys = world_cup_rankings.loc[home,'Volleys']        
        
        opp_rank = world_cup_rankings.loc[away, 'rank']
        opp_points = world_cup_rankings.loc[away, 'weighted_points']
        opp_rating = world_cup_rankings.loc[away,'Rating']
        opp_age = world_cup_rankings.loc[away,'Age']
        opp_weak_foot = world_cup_rankings.loc[away,'Weak_foot']
        opp_Skill_Moves = world_cup_rankings.loc[away,'Skill_Moves']
        opp_Ball_Control = world_cup_rankings.loc[away,'Ball_Control']      
        opp_Dribbling = world_cup_rankings.loc[away, 'Dribbling']
        opp_Marking = world_cup_rankings.loc[away, 'Marking']
        opp_Sliding_Tackle = world_cup_rankings.loc[away, 'Sliding_Tackle']
        opp_Standing_Tackle = world_cup_rankings.loc[away, 'Standing_Tackle']
        opp_Aggression = world_cup_rankings.loc[away, 'Aggression']
        opp_Reactions = world_cup_rankings.loc[away, 'Reactions']
        opp_Interceptions = world_cup_rankings.loc[away, 'Interceptions']
        opp_Vision = world_cup_rankings.loc[away, "Vision"]
        opp_Crossing = world_cup_rankings.loc[away, "Crossing"]
        opp_Short_Pass = world_cup_rankings.loc[away, "Short_Pass"]
        opp_Long_Pass = world_cup_rankings.loc[away, "Long_Pass"]                 
        opp_stamina = world_cup_rankings.loc[away,'Stamina']
        opp_Penalties = world_cup_rankings.loc[away,'Penalties']
        opp_Acceleration = world_cup_rankings.loc[away,'Acceleration']      
        opp_speed = world_cup_rankings.loc[away, 'Speed']
        opp_strength = world_cup_rankings.loc[away,'Strength']
        opp_balance = world_cup_rankings.loc[away,'Balance']
        opp_agility = world_cup_rankings.loc[away,'Agility']
        opp_jumping = world_cup_rankings.loc[away,'Jumping']
        opp_won = world_cup_rankings.loc[away,'Won']
        opp_drawn = world_cup_rankings.loc[away,'Drawn']
        opp_lost = world_cup_rankings.loc[away,'Lost']
        opp_average_points = world_cup_rankings.loc[away,'Average_points'] 
        opp_Heading = world_cup_rankings.loc[away,'Heading']
        opp_Shot_Power = world_cup_rankings.loc[away,'Shot_Power']
        opp_Finishing = world_cup_rankings.loc[away,'Finishing']
        opp_Long_Shots = world_cup_rankings.loc[away,'Long_Shots']
        opp_Curve = world_cup_rankings.loc[away,'Curve']
        opp_Freekick_Accuracy = world_cup_rankings.loc[away,'Freekick_Accuracy']
        opp_Volleys = world_cup_rankings.loc[away,'Volleys']
    
        row['average_rank'] = (home_rank + opp_rank) / 2
        row['rank_difference'] = home_rank - opp_rank
        row['point_difference'] = home_points - opp_points
        row['rating_difference'] = home_rating - opp_rating
        row['Age_difference'] = home_age - opp_age
        row['Weak_foot_difference'] = home_weak_foot - opp_weak_foot
        row['Skill_Moves_difference'] = home_Skill_Moves - opp_Skill_Moves
        row['Ball_Control_difference'] = home_rating - opp_rating        
        row["Dribbling_difference"] = home_Dribbling - opp_Dribbling
        row['Marking_difference'] = home_Marking - opp_Marking
        row['Sliding_Tackle_difference'] = home_Sliding_Tackle - opp_Sliding_Tackle
        row["Standing_Tackle_difference"] = home_Standing_Tackle - opp_Standing_Tackle
        row["Aggression_difference"] = home_Aggression - opp_Aggression
        row["Reactions_difference"] = home_Reactions - opp_Reactions
        row["Interceptions_difference"] = home_Interceptions - opp_Interceptions
        row["Vision_difference"] = home_Vision - opp_Vision
        row["Crossing_difference"] = home_Crossing - opp_Crossing
        row["Short_Pass_difference"] = home_Short_Pass - opp_Short_Pass
        row["Long_Pass_difference"] = home_Long_Pass - opp_Long_Pass
        row['Stamina_difference'] = home_stamina - opp_stamina
        row['Penalties_difference'] = home_Penalties - opp_Penalties
        row['Acceleration_difference'] = home_Acceleration - opp_Acceleration
        row['Speed_difference'] = home_speed - opp_speed
        row['Strength_difference'] = home_strength - opp_strength
        row['Balance_difference'] = home_balance - opp_balance
        row['Agility_difference'] = home_agility - opp_agility
        row['Jumping_difference'] = home_jumping - opp_jumping
        row['Won_difference'] = home_won - opp_won
        row['Drawn_difference'] = home_drawn - opp_drawn
        row['Lost_difference'] = home_lost - opp_lost
        row['Average_points_difference'] = home_average_points - opp_average_points
        row['Heading_difference'] = home_Heading - opp_Heading
        row['Shot_Power_difference'] = home_Shot_Power - opp_Shot_Power
        row['Finishing_difference'] = home_Finishing - opp_Finishing
        row['Long_Shots_difference'] = home_Long_Shots - opp_Long_Shots
        row['Curve_difference'] = home_Curve - opp_Curve
        row['Freekick_Accuracy_difference'] = home_Freekick_Accuracy - opp_Freekick_Accuracy
        row['Volleys_difference'] = home_Volleys - opp_Volleys
        
    
        
        home_win_prob = classifier_ann.predict_proba(row)[:,0][0]
        world_cup_matches.loc[home, 'total_prob'] += home_win_prob
        world_cup_matches.loc[away, 'total_prob'] += 1-home_win_prob
        
        points = 0
        if home_win_prob <= 0.5:
            print("{} wins with {:.2f}".format(away, 1-home_win_prob))
            world_cup_matches.loc[away, 'points'] += 3
        if home_win_prob > 0.5:
            points = 3
            world_cup_matches.loc[home, 'points'] += 3
            print("{} wins with {:.2f}".format(home, home_win_prob))


___Starting group H:___
Poland vs. Senegal: Poland wins with 0.60
Poland vs. Colombia: Poland wins with 0.56
Poland vs. Japan: Poland wins with 0.67
Senegal vs. Colombia: Colombia wins with 0.60
Senegal vs. Japan: Japan wins with 0.57
Colombia vs. Japan: Colombia wins with 0.53
___Starting group D:___
Argentina vs. Iceland: Argentina wins with 0.74
Argentina vs. Croatia: Argentina wins with 0.64
Argentina vs. Nigeria: Argentina wins with 0.82
Iceland vs. Croatia: Croatia wins with 0.63
Iceland vs. Nigeria: Iceland wins with 0.51
Croatia vs. Nigeria: Croatia wins with 0.67
___Starting group F:___
Germany vs. Mexico: Germany wins with 0.73
Germany vs. Sweden: Germany wins with 0.73
Germany vs. Korea Republic: Germany wins with 0.85
Mexico vs. Sweden: Sweden wins with 0.56
Mexico vs. Korea Republic: Mexico wins with 0.63
Sweden vs. Korea Republic: Sweden wins with 0.63
___Starting group E:___
Brazil vs. Switzerland: Brazil wins with 0.66
Brazil vs. Costa Rica: Brazil wins with 0.79
Brazil

In [29]:
huitieme = pd.read_excel("8e_de_final.xlsx")

huitieme = huitieme.dropna()
huitieme = huitieme.set_index("Home")
huitieme

,Group,Against
Home,,
Uruguay,A,Portugal
France,B,Argentina
Brazil,C,Mexico
Belgium,D,Japan
Spain,E,Russia
Croatia,F,Denmark
Sweden,G,Switzerland
Colombia,H,England
Portugal,A,Uruguay


In [30]:
margin = 0.05

huitieme['points'] = 0
huitieme['total_prob'] = 0

for group in set(huitieme['Group']):
    print('___Starting group {}:___'.format(group))
    for home, away in combinations(huitieme.query("Group == '{}'".format(group)).index, 2):
        print("{} vs. {}: ".format(home, away), end='')
        row = pd.DataFrame(np.array([[np.nan, 
                                      np.nan, 
                                      np.nan, 
                                      True, 
                                      np.nan,
                                      np.nan,
                                      np.nan,
                                      np.nan,
                                      np.nan,
                                      np.nan,
                                      np.nan,
                                      np.nan,
                                      np.nan,
                                      np.nan,
                                      np.nan,
                                      np.nan,
                                      np.nan,
                                      np.nan,
                                      np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan]]), columns= X_test.columns)
        
        
        home_rank = world_cup_rankings.loc[home, 'rank']
        home_points = world_cup_rankings.loc[home, 'weighted_points']
        home_rating = world_cup_rankings.loc[home,'Rating']
        home_age = world_cup_rankings.loc[home,'Age']
        home_weak_foot = world_cup_rankings.loc[home,'Weak_foot']
        home_Skill_Moves = world_cup_rankings.loc[home,'Skill_Moves']
        home_Ball_Control = world_cup_rankings.loc[home,'Ball_Control']
        home_Dribbling = world_cup_rankings.loc[home, 'Dribbling']
        home_Marking = world_cup_rankings.loc[home, 'Marking']
        home_Sliding_Tackle = world_cup_rankings.loc[home, 'Sliding_Tackle']
        home_Standing_Tackle = world_cup_rankings.loc[home, 'Standing_Tackle']
        home_Aggression = world_cup_rankings.loc[home, 'Aggression']
        home_Reactions = world_cup_rankings.loc[home, 'Reactions']
        home_Interceptions = world_cup_rankings.loc[home, 'Interceptions']
        home_Vision = world_cup_rankings.loc[home, "Vision"]
        home_Crossing = world_cup_rankings.loc[home, "Crossing"]
        home_Short_Pass = world_cup_rankings.loc[home, "Short_Pass"]
        home_Long_Pass = world_cup_rankings.loc[home, "Long_Pass"]                 
        home_stamina = world_cup_rankings.loc[home,'Stamina']
        home_Penalties = world_cup_rankings.loc[home,'Penalties']
        home_Acceleration = world_cup_rankings.loc[home,'Acceleration']        
        home_speed = world_cup_rankings.loc[home,'Speed']
        home_strength = world_cup_rankings.loc[home,'Strength']
        home_balance = world_cup_rankings.loc[home,'Balance']
        home_agility = world_cup_rankings.loc[home,'Agility']
        home_jumping = world_cup_rankings.loc[home,'Jumping']
        home_won = world_cup_rankings.loc[home,'Won']
        home_drawn = world_cup_rankings.loc[home,'Drawn']
        home_lost = world_cup_rankings.loc[home,'Lost']
        home_average_points = world_cup_rankings.loc[home,'Average_points']
        home_Heading = world_cup_rankings.loc[home,'Heading']
        home_Shot_Power = world_cup_rankings.loc[home,'Shot_Power']
        home_Finishing = world_cup_rankings.loc[home,'Finishing']
        home_Long_Shots = world_cup_rankings.loc[home,'Long_Shots']
        home_Curve = world_cup_rankings.loc[home,'Curve']
        home_Freekick_Accuracy = world_cup_rankings.loc[home,'Freekick_Accuracy']
        home_Volleys = world_cup_rankings.loc[home,'Volleys']        
        
        opp_rank = world_cup_rankings.loc[away, 'rank']
        opp_points = world_cup_rankings.loc[away, 'weighted_points']
        opp_rating = world_cup_rankings.loc[away,'Rating']
        opp_age = world_cup_rankings.loc[away,'Age']
        opp_weak_foot = world_cup_rankings.loc[away,'Weak_foot']
        opp_Skill_Moves = world_cup_rankings.loc[away,'Skill_Moves']
        opp_Ball_Control = world_cup_rankings.loc[away,'Ball_Control']      
        opp_Dribbling = world_cup_rankings.loc[away, 'Dribbling']
        opp_Marking = world_cup_rankings.loc[away, 'Marking']
        opp_Sliding_Tackle = world_cup_rankings.loc[away, 'Sliding_Tackle']
        opp_Standing_Tackle = world_cup_rankings.loc[away, 'Standing_Tackle']
        opp_Aggression = world_cup_rankings.loc[away, 'Aggression']
        opp_Reactions = world_cup_rankings.loc[away, 'Reactions']
        opp_Interceptions = world_cup_rankings.loc[away, 'Interceptions']
        opp_Vision = world_cup_rankings.loc[away, "Vision"]
        opp_Crossing = world_cup_rankings.loc[away, "Crossing"]
        opp_Short_Pass = world_cup_rankings.loc[away, "Short_Pass"]
        opp_Long_Pass = world_cup_rankings.loc[away, "Long_Pass"]                 
        opp_stamina = world_cup_rankings.loc[away,'Stamina']
        opp_Penalties = world_cup_rankings.loc[away,'Penalties']
        opp_Acceleration = world_cup_rankings.loc[away,'Acceleration']      
        opp_speed = world_cup_rankings.loc[away, 'Speed']
        opp_strength = world_cup_rankings.loc[away,'Strength']
        opp_balance = world_cup_rankings.loc[away,'Balance']
        opp_agility = world_cup_rankings.loc[away,'Agility']
        opp_jumping = world_cup_rankings.loc[away,'Jumping']
        opp_won = world_cup_rankings.loc[away,'Won']
        opp_drawn = world_cup_rankings.loc[away,'Drawn']
        opp_lost = world_cup_rankings.loc[away,'Lost']
        opp_average_points = world_cup_rankings.loc[away,'Average_points'] 
        opp_Heading = world_cup_rankings.loc[away,'Heading']
        opp_Shot_Power = world_cup_rankings.loc[away,'Shot_Power']
        opp_Finishing = world_cup_rankings.loc[away,'Finishing']
        opp_Long_Shots = world_cup_rankings.loc[away,'Long_Shots']
        opp_Curve = world_cup_rankings.loc[away,'Curve']
        opp_Freekick_Accuracy = world_cup_rankings.loc[away,'Freekick_Accuracy']
        opp_Volleys = world_cup_rankings.loc[away,'Volleys']
    
        row['average_rank'] = (home_rank + opp_rank) / 2
        row['rank_difference'] = home_rank - opp_rank
        row['point_difference'] = home_points - opp_points
        row['rating_difference'] = home_rating - opp_rating
        row['Age_difference'] = home_age - opp_age
        row['Weak_foot_difference'] = home_weak_foot - opp_weak_foot
        row['Skill_Moves_difference'] = home_Skill_Moves - opp_Skill_Moves
        row['Ball_Control_difference'] = home_rating - opp_rating        
        row["Dribbling_difference"] = home_Dribbling - opp_Dribbling
        row['Marking_difference'] = home_Marking - opp_Marking
        row['Sliding_Tackle_difference'] = home_Sliding_Tackle - opp_Sliding_Tackle
        row["Standing_Tackle_difference"] = home_Standing_Tackle - opp_Standing_Tackle
        row["Aggression_difference"] = home_Aggression - opp_Aggression
        row["Reactions_difference"] = home_Reactions - opp_Reactions
        row["Interceptions_difference"] = home_Interceptions - opp_Interceptions
        row["Vision_difference"] = home_Vision - opp_Vision
        row["Crossing_difference"] = home_Crossing - opp_Crossing
        row["Short_Pass_difference"] = home_Short_Pass - opp_Short_Pass
        row["Long_Pass_difference"] = home_Long_Pass - opp_Long_Pass
        row['Stamina_difference'] = home_stamina - opp_stamina
        row['Penalties_difference'] = home_Penalties - opp_Penalties
        row['Acceleration_difference'] = home_Acceleration - opp_Acceleration
        row['Speed_difference'] = home_speed - opp_speed
        row['Strength_difference'] = home_strength - opp_strength
        row['Balance_difference'] = home_balance - opp_balance
        row['Agility_difference'] = home_agility - opp_agility
        row['Jumping_difference'] = home_jumping - opp_jumping
        row['Won_difference'] = home_won - opp_won
        row['Drawn_difference'] = home_drawn - opp_drawn
        row['Lost_difference'] = home_lost - opp_lost
        row['Average_points_difference'] = home_average_points - opp_average_points
        row['Heading_difference'] = home_Heading - opp_Heading
        row['Shot_Power_difference'] = home_Shot_Power - opp_Shot_Power
        row['Finishing_difference'] = home_Finishing - opp_Finishing
        row['Long_Shots_difference'] = home_Long_Shots - opp_Long_Shots
        row['Curve_difference'] = home_Curve - opp_Curve
        row['Freekick_Accuracy_difference'] = home_Freekick_Accuracy - opp_Freekick_Accuracy
        row['Volleys_difference'] = home_Volleys - opp_Volleys
        
    
        
        home_win_prob = classifier_ann.predict_proba(row)[:,0][0]
        huitieme.loc[home, 'total_prob'] += home_win_prob
        huitieme.loc[away, 'total_prob'] += 1-home_win_prob
        
        points = 0
        if home_win_prob <= 0.5:
            print("{} wins with {:.2f}".format(away, 1-home_win_prob))
            huitieme.loc[away, 'points'] += 3
        if home_win_prob > 0.5:
            points = 3
            huitieme.loc[home, 'points'] += 3
            print("{} wins with {:.2f}".format(home, home_win_prob))


___Starting group H:___
Colombia vs. England: England wins with 0.66
___Starting group D:___
Belgium vs. Japan: Belgium wins with 0.69
___Starting group F:___
Croatia vs. Denmark: Denmark wins with 0.59
___Starting group E:___
Spain vs. Russia: Spain wins with 0.69
___Starting group C:___
Brazil vs. Mexico: Brazil wins with 0.67
___Starting group G:___
Sweden vs. Switzerland: Switzerland wins with 0.59
___Starting group A:___
Uruguay vs. Portugal: Portugal wins with 0.58
___Starting group B:___
France vs. Argentina: Argentina wins with 0.58


In [31]:
quart = pd.read_excel("4rt_de_final.xlsx")

quart = quart.dropna()
quart = quart.set_index("Home")
quart

,Group,Against
Home,,
France,A,Uruguay
Brazil,B,Belgium
Sweden,C,England
Russia,D,Croatia
Uruguay,A,France
Belgium,B,Brazil
England,C,Sweden
Croatia,D,Russia


In [32]:
quart['points'] = 0
quart['total_prob'] = 0

for group in set(quart['Group']):
    print('___Starting group {}:___'.format(group))
    for home, away in combinations(quart.query("Group == '{}'".format(group)).index, 2):
        print("{} vs. {}: ".format(home, away), end='')
        row = pd.DataFrame(np.array([[np.nan, 
                                      np.nan, 
                                      np.nan, 
                                      True, 
                                      np.nan,
                                      np.nan,
                                      np.nan,
                                      np.nan,
                                      np.nan,
                                      np.nan,
                                      np.nan,
                                      np.nan,
                                      np.nan,
                                      np.nan,
                                      np.nan,
                                      np.nan,
                                      np.nan,
                                      np.nan,
                                      np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan,
                                     np.nan]]), columns= X_test.columns)
        
        
        home_rank = world_cup_rankings.loc[home, 'rank']
        home_points = world_cup_rankings.loc[home, 'weighted_points']
        home_rating = world_cup_rankings.loc[home,'Rating']
        home_age = world_cup_rankings.loc[home,'Age']
        home_weak_foot = world_cup_rankings.loc[home,'Weak_foot']
        home_Skill_Moves = world_cup_rankings.loc[home,'Skill_Moves']
        home_Ball_Control = world_cup_rankings.loc[home,'Ball_Control']
        home_Dribbling = world_cup_rankings.loc[home, 'Dribbling']
        home_Marking = world_cup_rankings.loc[home, 'Marking']
        home_Sliding_Tackle = world_cup_rankings.loc[home, 'Sliding_Tackle']
        home_Standing_Tackle = world_cup_rankings.loc[home, 'Standing_Tackle']
        home_Aggression = world_cup_rankings.loc[home, 'Aggression']
        home_Reactions = world_cup_rankings.loc[home, 'Reactions']
        home_Interceptions = world_cup_rankings.loc[home, 'Interceptions']
        home_Vision = world_cup_rankings.loc[home, "Vision"]
        home_Crossing = world_cup_rankings.loc[home, "Crossing"]
        home_Short_Pass = world_cup_rankings.loc[home, "Short_Pass"]
        home_Long_Pass = world_cup_rankings.loc[home, "Long_Pass"]                 
        home_stamina = world_cup_rankings.loc[home,'Stamina']
        home_Penalties = world_cup_rankings.loc[home,'Penalties']
        home_Acceleration = world_cup_rankings.loc[home,'Acceleration']        
        home_speed = world_cup_rankings.loc[home,'Speed']
        home_strength = world_cup_rankings.loc[home,'Strength']
        home_balance = world_cup_rankings.loc[home,'Balance']
        home_agility = world_cup_rankings.loc[home,'Agility']
        home_jumping = world_cup_rankings.loc[home,'Jumping']
        home_won = world_cup_rankings.loc[home,'Won']
        home_drawn = world_cup_rankings.loc[home,'Drawn']
        home_lost = world_cup_rankings.loc[home,'Lost']
        home_average_points = world_cup_rankings.loc[home,'Average_points']
        home_Heading = world_cup_rankings.loc[home,'Heading']
        home_Shot_Power = world_cup_rankings.loc[home,'Shot_Power']
        home_Finishing = world_cup_rankings.loc[home,'Finishing']
        home_Long_Shots = world_cup_rankings.loc[home,'Long_Shots']
        home_Curve = world_cup_rankings.loc[home,'Curve']
        home_Freekick_Accuracy = world_cup_rankings.loc[home,'Freekick_Accuracy']
        home_Volleys = world_cup_rankings.loc[home,'Volleys']        
        
        opp_rank = world_cup_rankings.loc[away, 'rank']
        opp_points = world_cup_rankings.loc[away, 'weighted_points']
        opp_rating = world_cup_rankings.loc[away,'Rating']
        opp_age = world_cup_rankings.loc[away,'Age']
        opp_weak_foot = world_cup_rankings.loc[away,'Weak_foot']
        opp_Skill_Moves = world_cup_rankings.loc[away,'Skill_Moves']
        opp_Ball_Control = world_cup_rankings.loc[away,'Ball_Control']      
        opp_Dribbling = world_cup_rankings.loc[away, 'Dribbling']
        opp_Marking = world_cup_rankings.loc[away, 'Marking']
        opp_Sliding_Tackle = world_cup_rankings.loc[away, 'Sliding_Tackle']
        opp_Standing_Tackle = world_cup_rankings.loc[away, 'Standing_Tackle']
        opp_Aggression = world_cup_rankings.loc[away, 'Aggression']
        opp_Reactions = world_cup_rankings.loc[away, 'Reactions']
        opp_Interceptions = world_cup_rankings.loc[away, 'Interceptions']
        opp_Vision = world_cup_rankings.loc[away, "Vision"]
        opp_Crossing = world_cup_rankings.loc[away, "Crossing"]
        opp_Short_Pass = world_cup_rankings.loc[away, "Short_Pass"]
        opp_Long_Pass = world_cup_rankings.loc[away, "Long_Pass"]                 
        opp_stamina = world_cup_rankings.loc[away,'Stamina']
        opp_Penalties = world_cup_rankings.loc[away,'Penalties']
        opp_Acceleration = world_cup_rankings.loc[away,'Acceleration']      
        opp_speed = world_cup_rankings.loc[away, 'Speed']
        opp_strength = world_cup_rankings.loc[away,'Strength']
        opp_balance = world_cup_rankings.loc[away,'Balance']
        opp_agility = world_cup_rankings.loc[away,'Agility']
        opp_jumping = world_cup_rankings.loc[away,'Jumping']
        opp_won = world_cup_rankings.loc[away,'Won']
        opp_drawn = world_cup_rankings.loc[away,'Drawn']
        opp_lost = world_cup_rankings.loc[away,'Lost']
        opp_average_points = world_cup_rankings.loc[away,'Average_points'] 
        opp_Heading = world_cup_rankings.loc[away,'Heading']
        opp_Shot_Power = world_cup_rankings.loc[away,'Shot_Power']
        opp_Finishing = world_cup_rankings.loc[away,'Finishing']
        opp_Long_Shots = world_cup_rankings.loc[away,'Long_Shots']
        opp_Curve = world_cup_rankings.loc[away,'Curve']
        opp_Freekick_Accuracy = world_cup_rankings.loc[away,'Freekick_Accuracy']
        opp_Volleys = world_cup_rankings.loc[away,'Volleys']
    
        row['average_rank'] = (home_rank + opp_rank) / 2
        row['rank_difference'] = home_rank - opp_rank
        row['point_difference'] = home_points - opp_points
        row['rating_difference'] = home_rating - opp_rating
        row['Age_difference'] = home_age - opp_age
        row['Weak_foot_difference'] = home_weak_foot - opp_weak_foot
        row['Skill_Moves_difference'] = home_Skill_Moves - opp_Skill_Moves
        row['Ball_Control_difference'] = home_rating - opp_rating        
        row["Dribbling_difference"] = home_Dribbling - opp_Dribbling
        row['Marking_difference'] = home_Marking - opp_Marking
        row['Sliding_Tackle_difference'] = home_Sliding_Tackle - opp_Sliding_Tackle
        row["Standing_Tackle_difference"] = home_Standing_Tackle - opp_Standing_Tackle
        row["Aggression_difference"] = home_Aggression - opp_Aggression
        row["Reactions_difference"] = home_Reactions - opp_Reactions
        row["Interceptions_difference"] = home_Interceptions - opp_Interceptions
        row["Vision_difference"] = home_Vision - opp_Vision
        row["Crossing_difference"] = home_Crossing - opp_Crossing
        row["Short_Pass_difference"] = home_Short_Pass - opp_Short_Pass
        row["Long_Pass_difference"] = home_Long_Pass - opp_Long_Pass
        row['Stamina_difference'] = home_stamina - opp_stamina
        row['Penalties_difference'] = home_Penalties - opp_Penalties
        row['Acceleration_difference'] = home_Acceleration - opp_Acceleration
        row['Speed_difference'] = home_speed - opp_speed
        row['Strength_difference'] = home_strength - opp_strength
        row['Balance_difference'] = home_balance - opp_balance
        row['Agility_difference'] = home_agility - opp_agility
        row['Jumping_difference'] = home_jumping - opp_jumping
        row['Won_difference'] = home_won - opp_won
        row['Drawn_difference'] = home_drawn - opp_drawn
        row['Lost_difference'] = home_lost - opp_lost
        row['Average_points_difference'] = home_average_points - opp_average_points
        row['Heading_difference'] = home_Heading - opp_Heading
        row['Shot_Power_difference'] = home_Shot_Power - opp_Shot_Power
        row['Finishing_difference'] = home_Finishing - opp_Finishing
        row['Long_Shots_difference'] = home_Long_Shots - opp_Long_Shots
        row['Curve_difference'] = home_Curve - opp_Curve
        row['Freekick_Accuracy_difference'] = home_Freekick_Accuracy - opp_Freekick_Accuracy
        row['Volleys_difference'] = home_Volleys - opp_Volleys
        
    
        
        home_win_prob = classifier_ann.predict_proba(row)[:,0][0]
        quart.loc[home, 'total_prob'] += home_win_prob
        quart.loc[away, 'total_prob'] += 1-home_win_prob
        
        points = 0
        if home_win_prob <= 0.5:
            print("{} wins with {:.2f}".format(away, 1-home_win_prob))
            quart.loc[away, 'points'] += 3
        if home_win_prob > 0.5:
            points = 3
            quart.loc[home, 'points'] += 3
            print("{} wins with {:.2f}".format(home, home_win_prob))


___Starting group D:___
Russia vs. Croatia: Croatia wins with 0.66
___Starting group C:___
Sweden vs. England: England wins with 0.63
___Starting group A:___
France vs. Uruguay: France wins with 0.52
___Starting group B:___
Brazil vs. Belgium: Brazil wins with 0.64
